# Using Large Language Models

## Working with Text Models

In [1]:
# Imports

import os
from langchain_openai import OpenAI

Note, that LangChain automatically looks up for any environment variable with the name OPENAI_API_KEY automatically when making a connection to OpenAI.

In [2]:
# Initialize the Text Model

api_key = os.getenv("OPENAI_API_KEY")
llm = OpenAI(openai_api_key = api_key)

In [3]:
# Lets work on a simplest way to get a text autocomplete -
print(llm.invoke("Here is a fun fact about Pluto Planet :"))

 

Pluto was discovered in 1930 by American astronomer Clyde Tombaugh at the Lowell Observatory in Flagstaff, Arizona. It was the first planet to be discovered in the modern era.


In [4]:
# Lets generate a full output with mmore information
result = llm.generate(["Here is Fun Fact about Pluto : ",
                       "Here is a Fun Fact about Mars :"])

# to check the schema of the response
result.model_json_schema()

{'$defs': {'BaseMessage': {'additionalProperties': True,
   'description': 'Base abstract message class.\n\nMessages are the inputs and outputs of ChatModels.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'},
         {'additionalProperties': True, 'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'additionalProperties': True,
     'title': 'Additional Kwargs',
     'type': 'object'},
    'response_metadata': {'additionalProperties': True,
     'title': 'Response Metadata',
     'type': 'object'},
    'type': {'title': 'Type', 'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Id'}},
   'required': ['content', 'type'],
   'title': 'BaseMessage',
   'type': 'object'},
  'BaseMessageChunk': {'additionalProperties

In [5]:
# Check the detailed response
response = result.generations
print(response[0][0].text)




Pluto was discovered in 1930 by American astronomer Clyde Tombaugh. It was considered the ninth planet in our solar system until 2006 when it was reclassified as a "dwarf planet."


In [6]:
# Check the token usage and other details about output of the LLM

result.llm_output

{'token_usage': {'completion_tokens': 78,
  'total_tokens': 94,
  'prompt_tokens': 16},
 'model_name': 'gpt-3.5-turbo-instruct'}

## Working with Chat Models

In [7]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(openai_api_key = api_key)

In [8]:
# now lets ask a question
result = chat.invoke([HumanMessage(content="Can you tell me a fact about Earth ?")])
result

AIMessage(content='Sure! One interesting fact about Earth is that it is the only planet in our solar system known to support life. Its unique combination of atmosphere, temperature, and water make it the perfect environment for a wide variety of living organisms to thrive.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 16, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrOEx6dYa63rYVc5akXjygWg4Lcls', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e678f7da-99df-45d4-98d8-99727d695548-0', usage_metadata={'input_tokens': 16, 'output_tokens': 48, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_de

In [9]:
# Get the text response
result.content

'Sure! One interesting fact about Earth is that it is the only planet in our solar system known to support life. Its unique combination of atmosphere, temperature, and water make it the perfect environment for a wide variety of living organisms to thrive.'

In [10]:
# Lets try another
result = chat.generate([[SystemMessage(content="You are a University Professor"),
                         HumanMessage(content="Can you tell me a fact about Earth ?")]])

result.generations[0][0].text

'Sure! One interesting fact about Earth is that it is the only planet in our solar system known to have liquid water on its surface. This unique feature allows Earth to support a wide variety of life forms.'

- Temperature is a parameter that controls the “creativity” or randomness of the text generated by open ai llm. A higher temperature (e.g., 0.7) results in more diverse and creative output, while a lower temperature (e.g., 0.2) makes the output more deterministic and focused.

- In practice, temperature affects the probability distribution over the possible tokens at each step of the generation process. A temperature of 0 would make the model completely deterministic, always choosing the most likely token.

In [11]:
# We can add some extra parameters and args - here we are adding some extreme values
result = chat.invoke([HumanMessage(content="Can you tell me a fact about Earth ?")],
                     temperature=2, max_tokens=100)
result.content

"The coldest temperature ever recorded on Earth was minus 128.6 degrees Fahrenheit (minus 89.2 degrees Celsius) at Antarctica's Vostok station on July 21, 1983."

## Caching

Should only do this if the prompt is the exact same and the historical replies are okay to return.

In [13]:
import langchain
from langchain.cache import InMemoryCache

# Initialize the cache
langchain.llm_cache = InMemoryCache()

In [14]:
# The first time it is not in cache and hence might take little time
print(llm.invoke("Tell me fact about Mars"))



1. Mars is the fourth planet from the sun in our solar system, and is often referred to as the "Red Planet" due to its reddish appearance.

2. Mars has a thin atmosphere composed mainly of carbon dioxide, with traces of other gases such as nitrogen and argon.

3. The surface of Mars is covered in a variety of geological features, including volcanoes, canyons, and impact craters.

4. The largest volcano in the solar system, Olympus Mons, is located on Mars and stands at a height of 22 km (13.6 miles).

5. Mars has two small moons, Phobos and Deimos, which are irregularly shaped and are thought to be captured asteroids.

6. The average temperature on Mars is -80 degrees Fahrenheit (-62 degrees Celsius), making it a very cold planet.

7. The Martian day, or "sol," is slightly longer than Earth's, at 24 hours and 39 minutes.

8. Evidence suggests that Mars may have once had a much thicker atmosphere and flowing water on its surface, leading some scientists to believe that life may have o

In [15]:
# This response should come very fast
print(llm.invoke("Tell me fact about Mars"))



1. Mars is the fourth planet from the sun in our solar system, and is often referred to as the "Red Planet" due to its reddish appearance.

2. Mars has a thin atmosphere composed mainly of carbon dioxide, with traces of other gases such as nitrogen and argon.

3. The surface of Mars is covered in a variety of geological features, including volcanoes, canyons, and impact craters.

4. The largest volcano in the solar system, Olympus Mons, is located on Mars and stands at a height of 22 km (13.6 miles).

5. Mars has two small moons, Phobos and Deimos, which are irregularly shaped and are thought to be captured asteroids.

6. The average temperature on Mars is -80 degrees Fahrenheit (-62 degrees Celsius), making it a very cold planet.

7. The Martian day, or "sol," is slightly longer than Earth's, at 24 hours and 39 minutes.

8. Evidence suggests that Mars may have once had a much thicker atmosphere and flowing water on its surface, leading some scientists to believe that life may have o